In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/SINDy_tutorial')
path = '/content/drive/MyDrive/SINDy_tutorial/'

In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
os.chdir(path)
from SINDy_tools import calc_gradients, sequentially_thresholded_LSQ

In [28]:
f = h5py.File(path+'test.h5', 'r')
dx = f['dx'][()]
dt = f['dt'][()]
Qf = np.moveaxis(f['qtensor'][::4], 0, -1)
Qf = np.array([[Qf[0],Qf[1]],[Qf[1],-Qf[0]]])
uf = np.moveaxis(f['velocity'][::4], 0, -1)
Pf = np.moveaxis(f['pressure'][::4], 0, -1)
f.close()

In [51]:
Q = np.copy(Qf)# + np.random.normal(scale=1e-5, loc=0.0, size=Qf.shape)
Q[0,0] = -Q[1,1]
Q[0,1] = Q[1,0]
u = np.copy(uf)
P = np.copy(Pf)

In [52]:
'''All data is arranged as [...,x,y,t], where "..." corresponds to vector terms'''
lx, ly, lt = Q.shape[-3:]
tensorTerm = 0
numSamples = 100_000
numDataPoints = lx *ly *lt
sample = np.arange(numDataPoints)
np.random.shuffle(sample)
sample = sample[:numSamples]

In [53]:
lhs_term = '∇²u'
rhs_term_names = []
# term_names.append('∇²u')
rhs_term_names.append('∂ₜu')
rhs_term_names.append('u')
rhs_term_names.append('∇.Q')
rhs_term_names.append('Q.u')
rhs_term_names.append('(u.u)u')
rhs_term_names.append('∇(Q:Q)')
rhs_term_names.append('∇P')
# term_names.append('(∇.u)u')

rhs_term_names = np.array(rhs_term_names)

In [54]:
lap_u = calc_gradients(u, lap_order=[1], dh=(dx,dx))
grad_Q = calc_gradients(Q, grad_order=[1], dh=(dx,dx))
grad_P = calc_gradients(P, grad_order=[1], dh=(dx,dx))
dt_u = np.gradient(u, dt, axis=-1)


# LHS
lhs = np.copy(lap_u)[tensorTerm].flatten()[sample]

rhs = []
'''∂ₜu term'''
tmp = dt_u[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''u term'''
tmp = u[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''term_names.append('∇.Q')'''
tmp = np.einsum('aa...', grad_Q)[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''Q.u term'''
tmp = np.einsum('ia...,a...->i...', Q, u)[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''(u.u)u term'''
tmp = np.einsum('a...,a...->...', u, u)[None,...] *u
tmp = tmp[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''∇(Q:Q) term'''
tmp = 2 *np.einsum('iab...,ba...->i...', grad_Q, Q)
tmp = tmp[tensorTerm].flatten()[sample]
rhs.append(tmp)
'''∇P term'''
tmp = grad_P[tensorTerm].flatten()[sample]
rhs.append(tmp)

rhs = np.array(rhs).T

In [55]:
coeff, R2 = sequentially_thresholded_LSQ(rhs, lhs)

I = coeff != 0

print(coeff[2][I[2]])
print(rhs_term_names[I[2]])

[0.0197621  0.25748005 0.85943109]
['u' '∇.Q' '∇P']
